#!/usr/bin/env python<br># coding: utf-8

Author: Bao Cai

Course: Machine Learning for Descriptive Problems

Topic: Frequent Itemsets

Start Date: 2020-02-27

Last Save: 2020-02-28

1. Download the groceries.csv file from itslearning (also the file is in the folder homework inside the zip of this lecture)

2. Find the frequent pair of items (2-tuples) using the naïve, A-priori and PCY algorithms. For each of these compare the time of execution and results for supports s=10, 50, 100. Comment your results.

3. For the PCY algorithm, create up to 5 compact hash tables. What is  the difference in results and time of execution for 1,2,3,4 and 5 tables? Comment your results.

4. Find the final list of k-frequent items (k-tuples) for k=3,4 and 5. Experiment a bit and describe the best value for the support in each case. *Warning*: You can use any of the three algorithms, but be careful, because the algorithm can take too long if you don't chose it properly.

5. Using one of the results of the previous item, for one k (k=3,4 or 5) find the possible clusters using the 1-NN criteria. Comment your results.

In [34]:
# Libraries
import itertools
import numpy as np
from time import time
from datetime import datetime

In [2]:
# global variables
path_data = 'Data/groceries.csv'

In [56]:
# Function

def read_baskets(file, k=2, verbose=True):
    """
    Read a basket file, for each line is a basket.
    
    Parameters:
    ----------
    
    file: str
        Path to file.

    k: int
        Number of max items in a basket.
    
    verbose: boolean
        Choose to report on progress or not.
    
    Returns:
    -------

    basket_of_k: list
        List of baskets of size k.
    """
    with open(file) as f:
        baskets = f.readlines()

    basket_of_k = []
    n = 0
    for basket in baskets:
        items = basket.replace('\n', '').split(',')
        for itemset in itertools.combinations(items, k):
            basket_of_k.append(frozenset(itemset))
        if verbose:
            n += 1
            if n % 1000 == 0:
                print(n, 'baskets processed')
    return basket_of_k

def naive_frequency(baskets):
    """
    Return a dict of frequencies for each basket in given list.
    
    Parameters:
    ----------
    
    baskets: list
        List of baskets.
    
    Returns:
    -------

    basket_frequency: dict
        A frequency corresponds to each basket.
    """
    
    basket_frequency = {}
    for basket in baskets:
        if basket not in basket_frequency:
            basket_frequency[basket] = 0
        basket_frequency[basket] += 1
    return basket_frequency

def frequency_threshold(basket_frequency, s=100):
    """
    Return a dict of set frequencies exceed support threshold.
    
    Parameters:
    ----------
    
    basket_frequency: list
        A frequency corresponds to each basket.
    
    s: int
        Support threshold
    
    Returns:
    -------

    exceed_frequency: dict
        A dict of set frequencies exceed support threshold.
    """
    
    exceed_frequency = {}
    k = len(list(basket_frequency.keys())[0])
    for key, value in basket_frequency.items():
        if value >= s:
            exceed_frequency[key] = value
    print('{} itemsets of size {} with frequency exceed {}'.format(
        len(exceed_frequency), k, s
    ))
    return exceed_frequency

def a_piori_preset(
    k,
    s=100,
    larger_set=None,
    smaller_set=None
):
    """
    Return a union set of 2 given sets.
    
    Parameters:
    ----------
    
    k: int
        Basket's size.
    
    s: int
        Support threshold.
    
    larger_set: dict
        If there is a set that is larger than
        the other one, it has to be given to this
        variable.
    
    smaller_set: dict
        A dict of itemsets and their frequencies.
    
    Returns:
    -------

    frequent_preset: set
        A set of possible frequent itemsets.
    """
    
    basket_size = len(list(larger_set.keys())[0]) +\
                  len(list(smaller_set.keys())[0])
    if basket_size != k:
        print(
            'The given sets cannot be combined',
            'to produce a set of', k
        )
        return None
    frequent_preset = set([
        a.union(b)
        for a in larger_set.keys()
        for b in smaller_set.keys()
    ])
    return frequent_preset

def a_piori_filter(
    file,
    k,
    s=100,
    frequent_preset=None,
    read_baskets=read_baskets,
    naive_frequency=naive_frequency,
    frequency_threshold=frequency_threshold
):
    """
    Return a dict of set frequencies exceed support threshold.
    
    Parameters:
    ----------
    
    file: str
        Path to the data file.
    
    k: int
        Basket's size.
    
    s: int
        Support threshold.
        
    frequent_preset: set
        A set of possible frequent itemsets.
    
    read_baskets: function
    
    naive_frequency: function
    
    frequency_threshold: function
    
    Returns:
    -------

    frequency_threshold(filtered_set, s): dict
        A dict of set frequencies exceed support threshold.
    """
    
    if k == 1:
        return frequency_threshold(
            naive_frequency(
                read_baskets(file, k, False)
            ),
            s
        )
    filtered_set = {}
    for basket in read_baskets(file, k, False):
        if basket not in frequent_preset:
            continue
        if basket not in filtered_set:
            filtered_set[basket] = 0
        filtered_set[basket] += 1
        
    return frequency_threshold(filtered_set, s)

def PCY_hash(
    file,
    k,
    n_hash=2,
    s=100,
    read_baskets=read_baskets
):
    """
    Return list of hashing tables.
    
    Parameters:
    ----------
    
    file: str
        Path to the data file.
    
    k: int
        Basket's size.
        
    n_hash: int
        Number of hash table.
    
    s: int
        Support threshold.
    
    read_baskets: function
    
    Returns:
    -------

    hash_tables: list
        A list of hashing tables.
    """
    
    hash_tables = []
    for i in range(n_hash):
        max_hash = 5*1000000 + i*1024
        hash_tables.append([np.zeros((max_hash,), dtype=int), max_hash])
    
    for key in read_baskets(file, k, False):
        for hash_table, max_hash in hash_tables:
            hash_table[hash(key)%max_hash] += 1
    for i in range(len(hash_tables)):
        hash_tables[i][0] = set(np.where(hash_tables[i][0] >= s)[0])
    return hash_tables

def PCY_filter(
    file,
    k,
    s=100,
    hash_tables=None,
    frequent_preset=None,
    read_baskets=read_baskets,
    frequency_threshold=frequency_threshold
):
    """
    Return a dict of set frequencies exceed support threshold.
    
    Parameters:
    ----------
    
    file: str
        Path to the data file.
    
    k: int
        Basket's size.
    
    s: int
        Support threshold.
    
    hash_tables: list
        A list of hashing tables.
        
    frequent_preset: set
        A set of possible frequent itemsets.
    
    read_baskets: function
    
    frequency_threshold: function
    
    Returns:
    -------

    frequency_threshold(filtered_set, s): dict
        A dict of set frequencies exceed support threshold.
    """

    filtered_set = {}
    for basket in read_baskets(file, k, False):
        if basket not in frequent_preset:
            continue
        accept = True
        for hash_table, max_hash in hash_tables:
            hashed = hash(basket)%max_hash
            if hashed not in hash_table:
                accept = False
                break
        if not accept:
            continue
        if basket not in filtered_set:
            filtered_set[basket] = 0
        filtered_set[basket] += 1
    return frequency_threshold(filtered_set, s)

class Node:
    """Node class for the linked list."""

    def __init__(self, value, nxt=None):
        """Initialize the node."""
        self.value = value
        self.next = nxt

def clustering(lst):
    """
    Return a list of clusters.
    
    Parameters:
    ----------
    
    lst: list
        A list of pairs.
    
    Returns:
    -------

    lst_cluster: list
        A list of clusters.
    """
    
    lst_element = []
    lst_head = []
    lst_node = []
    
    i, j = lst[0]
    lst_element = lst_element + [i, j]
    lst_node.append(Node(i))
    lst_head.append(lst_node[-1])
    lst_node.append(Node(j))
    lst_node[-2].next = lst_node[-1]
    n = 1
    
    t1 = time()
    print('Start checking pairs')
    c = 0
    for i, j in lst[1:]:
        if i in lst_element:
            lst_element.append(j)
            lst_node.append(Node(j))
            n += 1
            k = lst_element.index(i)
            lst_node[k].next, lst_node[n].next = lst_node[n], lst_node[k].next
        elif j in lst_element:
            lst_element.append(i)
            lst_node.append(Node(i))
            n += 1
            k = lst_element.index(j)
            lst_node[k].next, lst_node[n].next = lst_node[n], lst_node[k].next
        else:
            lst_element.append(i)
            lst_element.append(j)
            lst_node.append(Node(i))
            lst_head.append(lst_node[-1])
            lst_node.append(Node(j))
            lst_node[-2].next = lst_node[-1]
            n += 2
        c += 1
        if c%100000 == 0:
            print('100000 pairs checked in {} seconds'.format(time() - t1))
    
    lst_cluster = []
    for head in lst_head:
        temp = []
        while head:
            temp.append(head.value)
            head = head.next
        lst_cluster.append(temp)
    return lst_cluster

In [4]:
with open('Data/groceries.csv') as f:
    baskets = f.readlines()

baskets = [
    frozenset(
        basket.replace('\n', '').split(',')
    ) for basket in baskets
]

In [5]:
baskets = read_baskets(path_data, k=2)

1000 baskets processed
2000 baskets processed
3000 baskets processed
4000 baskets processed
5000 baskets processed
6000 baskets processed
7000 baskets processed
8000 baskets processed
9000 baskets processed


In [6]:
%%time
naive_itemsets = naive_frequency(baskets)
naive_100_threshold = frequency_threshold(naive_itemsets)

207 itemsets of size 2 with frequency exceed 100
CPU times: user 83.7 ms, sys: 3.45 ms, total: 87.2 ms
Wall time: 83.6 ms


In [7]:
%%time
naive_itemsets = naive_frequency(baskets)
naive_50_threshold = frequency_threshold(naive_itemsets, 50)

605 itemsets of size 2 with frequency exceed 50
CPU times: user 85.5 ms, sys: 3.07 ms, total: 88.6 ms
Wall time: 85 ms


In [8]:
%%time
naive_itemsets = naive_frequency(baskets)
naive_20_threshold = frequency_threshold(naive_itemsets, 20)

1674 itemsets of size 2 with frequency exceed 20
CPU times: user 75.2 ms, sys: 7.01 ms, total: 82.2 ms
Wall time: 78.5 ms


In [9]:
%%time
naive_itemsets = naive_frequency(baskets)
naive_10_threshold = frequency_threshold(naive_itemsets, 10)

2981 itemsets of size 2 with frequency exceed 10
CPU times: user 74.1 ms, sys: 7.99 ms, total: 82.1 ms
Wall time: 78.6 ms


In [10]:
%%time
naive_k3_itemsets = naive_frequency(read_baskets(path_data, 3, False))
naive_k3_100_threshold = frequency_threshold(naive_k3_itemsets, 100)

31 itemsets of size 3 with frequency exceed 100
CPU times: user 1.02 s, sys: 138 ms, total: 1.16 s
Wall time: 1.15 s


In [11]:
%%time
naive_k3_itemsets = naive_frequency(read_baskets(path_data, 3, False))
naive_k3_20_threshold = frequency_threshold(naive_k3_itemsets, 20)

1991 itemsets of size 3 with frequency exceed 20
CPU times: user 1.19 s, sys: 44.6 ms, total: 1.23 s
Wall time: 1.23 s


In [12]:
%%time
naive_k4_itemsets = naive_frequency(read_baskets(path_data, 4, False))
naive_k4_20_threshold = frequency_threshold(naive_k4_itemsets, 20)

395 itemsets of size 4 with frequency exceed 20
CPU times: user 3.58 s, sys: 186 ms, total: 3.77 s
Wall time: 3.76 s


In [13]:
%%time
naive_k5_itemsets = naive_frequency(read_baskets(path_data, 5, False))
naive_k5_20_threshold = frequency_threshold(naive_k5_itemsets, 20)

16 itemsets of size 5 with frequency exceed 20
CPU times: user 10.1 s, sys: 1.32 s, total: 11.5 s
Wall time: 11.4 s


### A-Piori

In [14]:
apiori_k1_s100 = a_piori_filter(path_data, 1)
apiori_k1_s50 = a_piori_filter(path_data, 1, 50)
apiori_k1_s10 = a_piori_filter(path_data, 1, 10)

88 itemsets of size 1 with frequency exceed 100
120 itemsets of size 1 with frequency exceed 50
157 itemsets of size 1 with frequency exceed 10


In [15]:
apiori_k2_s100 = a_piori_preset(
    2,
    100,
    apiori_k1_s100,
    apiori_k1_s100
)

In [16]:
%%time
apiori_k2_s100 = a_piori_filter(
    path_data,
    2,
    100,
    apiori_k2_s100
)

207 itemsets of size 2 with frequency exceed 100
CPU times: user 273 ms, sys: 5.93 ms, total: 279 ms
Wall time: 275 ms


In [17]:
apiori_k2_s50 = a_piori_preset(
    2,
    50,
    apiori_k1_s50,
    apiori_k1_s50
)

In [18]:
%%time
apiori_k2_s50 = a_piori_filter(
    path_data,
    2,
    50,
    apiori_k2_s50
)

605 itemsets of size 2 with frequency exceed 50
CPU times: user 268 ms, sys: 2.92 ms, total: 271 ms
Wall time: 268 ms


In [19]:
apiori_k2_s10 = a_piori_preset(
    2,
    10,
    apiori_k1_s10,
    apiori_k1_s10
)

In [20]:
%%time
apiori_k2_s10 = a_piori_filter(
    path_data,
    2,
    10,
    apiori_k2_s10
)

2981 itemsets of size 2 with frequency exceed 10
CPU times: user 288 ms, sys: 3 ms, total: 291 ms
Wall time: 288 ms


In [21]:
apiori_k3_s100 = a_piori_preset(
    3,
    100,
    apiori_k2_s100,
    apiori_k1_s100
)

In [22]:
%%time
apiori_k3_s100 = a_piori_filter(
    path_data,
    3,
    100,
    apiori_k3_s100
)

31 itemsets of size 3 with frequency exceed 100
CPU times: user 579 ms, sys: 21.1 ms, total: 600 ms
Wall time: 596 ms


In [23]:
apiori_k1_s20 = a_piori_filter(path_data, 1, 20)
apiori_k2_s20 = a_piori_preset(
    2,
    20,
    apiori_k1_s20,
    apiori_k1_s20
)
apiori_k2_s20 = a_piori_filter(
    path_data,
    2,
    20,
    apiori_k2_s20
)

apiori_k3_s20 = a_piori_preset(
    3,
    20,
    apiori_k2_s20,
    apiori_k1_s20
)

apiori_k4_s20 = a_piori_preset(
    4,
    20,
    apiori_k2_s20,
    apiori_k2_s20
)

147 itemsets of size 1 with frequency exceed 20
1674 itemsets of size 2 with frequency exceed 20


In [24]:
%%time
apiori_k3_s20 = a_piori_filter(
    path_data,
    3,
    20,
    apiori_k3_s20
)

1991 itemsets of size 3 with frequency exceed 20
CPU times: user 10.2 s, sys: 19.1 s, total: 29.3 s
Wall time: 1min


In [25]:
apiori_k5_s20 = a_piori_preset(
    5,
    20,
    apiori_k3_s20,
    apiori_k2_s20
)

In [26]:
%%time
apiori_k4_s20 = a_piori_filter(
    path_data,
    4,
    20,
    apiori_k4_s20
)

395 itemsets of size 4 with frequency exceed 20
CPU times: user 3.66 s, sys: 0 ns, total: 3.66 s
Wall time: 3.79 s


In [27]:
%%time
apiori_k5_s20 = a_piori_filter(
    path_data,
    5,
    20,
    apiori_k5_s20
)

16 itemsets of size 5 with frequency exceed 20
CPU times: user 1min 25s, sys: 5min 32s, total: 6min 58s
Wall time: 18min 32s


Basket size 2 and 3 seems large so I checked with Naive method above and it's checked out.

Also, I think the more advance algorithm will benefit when the number of items in a single basket is relatively large or equal to the number of unique items. The reason behind that statement is that for this dataset, there's not much 5 items basket (I guess, I didn't open many of them), so naive method can just run through them and count pretty easily. I did also suspect my function made it loop too many times on the read length and all so I splited them into 2 part, the filtering part alone took quite long as well. I ran out of ideas.

Then the support makes the time goes up the lower it gets.

### PCY

In [38]:
%%time
PCY_k2_s100 = a_piori_preset(
    2,
    100,
    apiori_k1_s100,
    apiori_k1_s100
)
PCY_k2_100 = PCY_filter(
    path_data,
    2,
    s=100,
    hash_tables=PCY_hash(
        path_data,
        2,
        s=100
    ),
    frequent_preset=a_piori_preset(
        2,
        100,
        apiori_k1_s100,
        apiori_k1_s100
    )
)

207 itemsets of size 2 with frequency exceed 100
CPU times: user 1 s, sys: 0 ns, total: 1 s
Wall time: 998 ms


In [39]:
%%time
PCY_k2_s50 = a_piori_preset(
    2,
    50,
    apiori_k1_s50,
    apiori_k1_s50
)
PCY_k2_50 = PCY_filter(
    path_data,
    2,
    s=50,
    hash_tables=PCY_hash(
        path_data,
        2,
        s=50
    ),
    frequent_preset=a_piori_preset(
        2,
        50,
        apiori_k1_s50,
        apiori_k1_s50
    )
)

605 itemsets of size 2 with frequency exceed 50
CPU times: user 1.11 s, sys: 0 ns, total: 1.11 s
Wall time: 1.1 s


In [40]:
%%time
PCY_k2_s10 = a_piori_preset(
    2,
    10,
    apiori_k1_s10,
    apiori_k1_s10
)
PCY_k2_10 = PCY_filter(
    path_data,
    2,
    s=10,
    hash_tables=PCY_hash(
        path_data,
        2,
        s=10
    ),
    frequent_preset=a_piori_preset(
        2,
        10,
        apiori_k1_s10,
        apiori_k1_s10
    )
)

2981 itemsets of size 2 with frequency exceed 10
CPU times: user 3.9 s, sys: 0 ns, total: 3.9 s
Wall time: 3.87 s


In [41]:
%%time
PCY_k2_s100 = a_piori_preset(
    2,
    100,
    apiori_k1_s100,
    apiori_k1_s100
)
PCY_k2_100 = PCY_filter(
    path_data,
    2,
    s=100,
    hash_tables=PCY_hash(
        path_data,
        2,
        n_hash=3,
        s=100
    ),
    frequent_preset=a_piori_preset(
        2,
        100,
        apiori_k1_s100,
        apiori_k1_s100
    )
)

207 itemsets of size 2 with frequency exceed 100
CPU times: user 1.2 s, sys: 0 ns, total: 1.2 s
Wall time: 1.2 s


In [42]:
%%time
PCY_k2_s100 = a_piori_preset(
    2,
    100,
    apiori_k1_s100,
    apiori_k1_s100
)
PCY_k2_100 = PCY_filter(
    path_data,
    2,
    s=100,
    hash_tables=PCY_hash(
        path_data,
        2,
        n_hash=4,
        s=100
    ),
    frequent_preset=a_piori_preset(
        2,
        100,
        apiori_k1_s100,
        apiori_k1_s100
    )
)

207 itemsets of size 2 with frequency exceed 100
CPU times: user 1.3 s, sys: 0 ns, total: 1.3 s
Wall time: 1.29 s


In [43]:
%%time
PCY_k2_s100 = a_piori_preset(
    2,
    100,
    apiori_k1_s100,
    apiori_k1_s100
)
PCY_k2_100 = PCY_filter(
    path_data,
    2,
    s=100,
    hash_tables=PCY_hash(
        path_data,
        2,
        n_hash=5,
        s=100
    ),
    frequent_preset=a_piori_preset(
        2,
        100,
        apiori_k1_s100,
        apiori_k1_s100
    )
)

207 itemsets of size 2 with frequency exceed 100
CPU times: user 1.41 s, sys: 0 ns, total: 1.41 s
Wall time: 1.41 s


More hash tables means more comparison so in the process itself, the more hash table the longer it takes. Still, I think the reason why it took too long is still because of the small basket in the data

### Clustering

In [53]:
# I'll do clustering on this
print(len(apiori_k2_s20), 'frequent sets')
list(apiori_k2_s20.keys())[:20]

1674 frequent sets


[frozenset({'citrus fruit', 'semi-finished bread'}),
 frozenset({'citrus fruit', 'margarine'}),
 frozenset({'margarine', 'semi-finished bread'}),
 frozenset({'tropical fruit', 'yogurt'}),
 frozenset({'coffee', 'tropical fruit'}),
 frozenset({'coffee', 'yogurt'}),
 frozenset({'pip fruit', 'yogurt'}),
 frozenset({'cream cheese', 'pip fruit'}),
 frozenset({'cream cheese', 'yogurt'}),
 frozenset({'other vegetables', 'whole milk'}),
 frozenset({'condensed milk', 'other vegetables'}),
 frozenset({'long life bakery product', 'other vegetables'}),
 frozenset({'condensed milk', 'whole milk'}),
 frozenset({'long life bakery product', 'whole milk'}),
 frozenset({'butter', 'whole milk'}),
 frozenset({'whole milk', 'yogurt'}),
 frozenset({'rice', 'whole milk'}),
 frozenset({'butter', 'yogurt'}),
 frozenset({'rice', 'yogurt'}),
 frozenset({'UHT-milk', 'other vegetables'})]

In [61]:
for cluster in clustering(list(apiori_k2_s20.keys())):
    print(cluster[:20], end='\n\n')

Start checking pairs
['semi-finished bread', 'pip fruit', 'soda', 'newspapers', 'root vegetables', 'sausage', 'whipped/sour cream', 'shopping bags', 'fruit/vegetable juice', 'curd', 'citrus fruit', 'potted plants', 'bottled beer', 'grapes', 'herbs', 'pickled vegetables', 'shopping bags', 'salty snack', 'baking powder', 'soda']

['tropical fruit', 'specialty bar', 'chewing gum', 'roll products', 'mustard', 'cat food', 'canned beer', 'dessert', 'ice cream', 'specialty chocolate', 'potted plants', 'long life bakery product', 'canned fish', 'grapes', 'seasonal products', 'UHT-milk', 'frozen vegetables', 'beverages', 'red/blush wine', 'beef']

['whole milk', 'house keeping products', 'liver loaf', 'kitchen towels', 'popcorn', 'instant coffee', 'jam', 'female sanitary products', 'chewing gum', 'turkey', 'sauces', 'white wine', 'chocolate marshmallow', 'ice cream', 'Instant food products', 'sweet spreads', 'curd cheese', 'mayonnaise', 'cleaner', 'specialty cheese']

['pastry', 'canned fish', 

So there are about 6 clusters:

- The first cluster looks like a healthy life style
- The second one, not so much, a lot of sugar in take
- The third one belongs to the ladies

The rest are quite random to me. Also there are some overlapping items that appeared in more than 1 cluster like `meat` which also make sense.